In [1]:
import sqlalchemy
import pyodbc
from urllib.parse import quote
import pandas as pd
from tqdm import tqdm
engine=sqlalchemy.create_engine('mssql+pyodbc://SAL_USR:%s@172.16.20.46:1433/HDM_DB?driver=ODBC+Driver+17+for+SQL+Server' % quote('HDM@123_'))

In [2]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [3]:
pans_list = ['ABJPA6843P','BGCPS0192L','ACNPP3358F','BCYPT2965A','ANEPW7396G','CAZPC7158J','DKJPR8881J','BPUPN9580E','ADPPH6281C','DDQPK1853M','BMNPA5597E','APOPG2789R']

In [4]:
def remove_list_brackets(list_):
    str_no_list = str(list_)
    str_no_list = str_no_list.replace('[','(').replace(']',')')
    return str_no_list

In [6]:
import os

In [7]:
for chunk in tqdm(list(chunks(pans_list,10))):
        pans = remove_list_brackets(chunk)
        ctes = ['cte1','cte2','cte3','cte4']
        for cte in ctes:
            query = f'''with cte1 as (select batchid,fiureid from [FINCORE_BACKUP_CC].[BATCH] where batchid between 1804010000 and 2303319999 and reporttype='CTR' and reportformattype='ARF' and batchtype<>'D' and batchid not in (select distinct originalbatchid from [FINCORE_BACKUP_CC].[BATCH] where batchid between 1804010000 and 2303319999)),

            cte2 as (select batchid, rptsernum, trim(accountnum) accountnum, pan, mobile,commstatecode from [FINCORE_BACKUP_CC].[ARFLPE]
            where batchid in (select batchid from cte1) and pan in {pans}
            union
            select batchid, rptsernum, trim(accountnum) accountnum, pan, mobile,commstatecode from [FINCORE_BACKUP_CC].[ARFINP]
            where batchid in (select batchid from cte1) and pan in {pans}),
            cte3 as (select pan, batchid, rptsernum, accountnum,sum(case when debitcredit='C' then amount end)credit_amount,sum(case when debitcredit='D' then amount end)debit_amount from(
            select pan, t.batchid,t.rptsernum,trim(t.accountnum)accountnum,debitcredit,amount from [FINCORE_BACKUP_CC].[ARFTRN] t inner join (select distinct pan, batchid, rptsernum,trim(accountnum)accountnum from cte2) i
            on t.batchid=i.batchid and t.rptsernum=i.rptsernum and trim(t.accountnum)=trim(i.accountnum))z
            group by pan, batchid, rptsernum, accountnum),
            cte4 as (select pan, sum(credit_amount)credit_amount, sum(debit_amount)debit_amount from cte3 group by pan), 
            cte5 as ( select pan, b.batchid, b.rptsernum, trim(b.accountnum)accountnum,statecode from [FINCORE_BACKUP_CC].[ARFBRC] b inner join 
            (select distinct pan, batchid, rptsernum,trim(accountnum)accountnum from cte2) i on b.batchid=i.batchid and b.rptsernum=i.rptsernum and trim(b.accountnum)=trim(i.accountnum))
            select * from {cte}'''
            results = pd.read_sql(query,engine)
            results.to_csv(f"C:\\Users\\SAL007\\Desktop\\15-06-2023_CTR\\{cte}.csv",index=False,mode='a',header= not os.path.exists(f"C:\\Users\\SAL007\\Desktop\\15-06-2023_CTR\\{cte}.csv")) 

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [01:47<00:00, 53.72s/it]
